In [1]:
from dotenv import load_dotenv
load_dotenv()

from ecmwf.opendata import Client
from datetime import timedelta
import pandas as pd
import time
import logging
from pymongo import MongoClient

from pipeline.downloader import Downloader
from pipeline.orchestrator import PipelineOrchestrator
from pipeline.runner import PipelineRunner
from db.repo import ResultsRepository
from db.mongo import MongoWrapper

from pipeline.config import settings

from datetime import datetime

from model.forecast import base_vs_forecast, forecast_vs_forecast, full_forecast

In [7]:
steps = [i for i in range(0, 144, 3)] + [j for j in range(144, 361, 6)] #get all steps
time = 0 if datetime.now().hour < 18 else 12 #get 00z forecast if we are before 18z, else get 12z forecast
current_date = datetime.now().date() #select current date forecast
var = '2t' #forecast returned variables

request = {
    'date': current_date,
    'time': 0,
    'type': "fc",
    'stream': 'oper',
    'step': steps,
    'param': var,
}       
d = Downloader(request=request, download_dir=settings.DOWNLOAD_DIR)
filepath, run_datetime = d.download()

In [ ]:
latest_run_date = d.check_latest_available()
previous_run_date = pd.Timestamp(latest_run_date) - pd.Timedelta(hours=12)
latest_run_date

In [19]:
previous_run_date

Timestamp('2026-02-01 12:00:00')

In [20]:
horizons = [(previous_run_date.date() + timedelta(days=1), previous_run_date.date() + timedelta(days=3), 'Day 1-3'),
                    (previous_run_date.date() + timedelta(days=4), previous_run_date.date() + timedelta(days=7), 'Day 4-7'),
                    (previous_run_date.date() + timedelta(days=8), previous_run_date.date() + timedelta(days=14), 'Day 8-14')]

base_forecast = base_vs_forecast(filepath, settings.BASE_FILE_PATH, horizons)

Ignoring index file 'data/current.grib2.5b7b6.idx' older than GRIB file


In [21]:
mongo = MongoWrapper(settings.MONGO_URI, settings.MONGO_DB)
repo = ResultsRepository(mongo_client=mongo, repo=settings.MONGO_COLLECTION)

repo.insert_results(base_forecast)

In [ ]:
from datetime import datetime, timezone

def is_run_release_time():
    now = datetime.now(timezone.utc)
    minutes = now.hour * 60 + now.minute

    #00z window: 6am to 9am UTC
    if 6 * 60 <= minutes < 9 * 60:
        return True

    #00z window: 6pm to 9pm UTC
    if 18 * 60 <= minutes < 21 * 60:
        return True

    return False